# Get All The Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
from IPython.display import display, Markdown
import plotly.graph_objects as go
import plotly.io as pio
import os
from datetime import datetime

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import utils with different aliases
from src.utils import csv_exporter as csv_utils
from src.utils import validation as val_utils
from src.utils import transformations as trans_utils
from src.utils import data_merger as merge_utils
from src.utils import config_validator as config_utils
from src.utils import metrics as metric_utils

# Read the CSV files using our utility function and alining start dates
data_dir = project_root / 'data'
sprds = csv_utils.read_csv_to_df(data_dir / 'sprds_data.csv', fill='ffill', start_date_align='yes')
derv = csv_utils.read_csv_to_df(data_dir / 'derv_data.csv', fill='ffill', start_date_align='yes')
er_ytd = csv_utils.read_csv_to_df(data_dir / 'er_ytd_data.csv', fill='ffill', start_date_align='yes')

# Convert er_ytd table into credit unlevered indices
credit_unlevered_index = trans_utils.convert_er_ytd_to_index(er_ytd)
credit_unlevered_index = credit_unlevered_index[['cad_ig_oas_index', 'us_ig_oas_index','us_hy_oas_index']]
# Create levered index using get_er_index
credit_levered_index = trans_utils.get_er_index(credit_unlevered_index, cost_of_borrow=40, leverage=3.0)

# Get just the columns i want from sprds
sprds_selected = sprds[['cad_ig_oas', 'us_ig_oas', 'us_hy_oas']]

# Merge sprds with credit_unlevered_index
sprds_credit_unlevered_index = merge_utils.merge_dfs(sprds_selected, credit_unlevered_index, fill='ffill', start_date_align='yes')

2024-12-09 16:32:16,832 - WARNING - Cannot infer frequency. Attempting to resample to daily frequency.
2024-12-09 16:32:16,835 - INFO - Inferred frequency: D


# Generating Interesting Visuals

In [2]:
# Resample the data to monthly frequency, taking the last observation of each month  
sprds_credit_unlevered_index_monthly_last = sprds_credit_unlevered_index.resample('ME').last()  
  
# Calculate the 12-period forward percentage change for 'cad_ig_oas_index'  
sprds_credit_unlevered_index_monthly_last['cad_ig_oas_index_forward_pct_change_12'] = sprds_credit_unlevered_index_monthly_last['cad_ig_oas_index'].pct_change(periods=12).shift(-12) * 100  
  
# Identify the last data point  
last_data_point = sprds_credit_unlevered_index_monthly_last.iloc[-1]  

# Create a scatter plot to show the relationship with hover information including the date  
fig = go.Figure()  

# Plot all data points  
fig.add_trace(go.Scatter(  
    x=sprds_credit_unlevered_index_monthly_last['cad_ig_oas'],  
    y=sprds_credit_unlevered_index_monthly_last['cad_ig_oas_index_forward_pct_change_12'],  
    mode='markers',  
    marker=dict(color='cyan', size=8),  
    text=sprds_credit_unlevered_index_monthly_last.index.strftime('%Y-%m-%d'),  
    hovertemplate='<b>Date:</b> %{text}<br>' +  
                  '<b>CAD IG OAS:</b> %{x}<br>' +  
                  '<b>Forward % Change:</b> %{y:.2f}%<extra></extra>',  
    showlegend=False  
))  

# Update layout with responsive settings  
fig.update_layout(  
    title='Relationship Between Starting CAD Spread Levels & 12M Forward Unlevered Credit Returns',  
    xaxis_title='CAD IG Starting Sprdead',  
    yaxis_title='12M Forward Unlevered Credit Returns (%)',  
    template='plotly_dark',  
    autosize=True,  
    height=600,  
    margin=dict(l=50, r=50, t=50, b=50),  
)  

# Configure for notebook display  
config = {  
    'responsive': True,  
    'displayModeBar': True,  
    'displaylogo': False,  
}  

# Show the plot with config  
fig.show(config=config)

# Create plots directory if it doesn't exist
plots_dir = os.path.join(os.getcwd(), 'plots')
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Generate filename with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'cad_ig_oas_returns_relationship_{timestamp}.html'
filepath = os.path.join(plots_dir, filename)

# Save the plot
fig.write_html(
    filepath,
    include_plotlyjs=True,
    full_html=True,
    config=config
)

print(f"Plot saved to: {filepath}")

Plot saved to: c:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\windsurf_bloomberg\notebooks\plots\cad_ig_oas_returns_relationship_20241209_163217.html
